# Translator

In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import tokenizers
import unicodedata
import zipfile

In [3]:
en_tokenizer = tokenizers.Tokenizer.from_file("en_tokenizer.json")
pl_tokenizer = tokenizers.Tokenizer.from_file("pl_tokenizer.json")

In [5]:
# Normalize text
# each line of the file is in the format "<english>\t<french>"
# We convert text to lowercase, normalize unicode (UFKC)
def normalize(line):
    """Normalize a line of text and split into two at the tab character"""
    line = unicodedata.normalize("NFKC", line.strip().lower())
    parts = line.split("\t")
    if len(parts) < 2:
        return None  # pomiń niepoprawne linie

    # niektóre linie mają więcej niż jedno tłumaczenie – weź tylko pierwsze dwa
    eng, pl = parts[0], parts[1]

    return eng.strip(), pl.strip()

text_pairs = []
with zipfile.ZipFile("pol-eng.zip", "r") as zip_ref:
    for line in zip_ref.read("pol.txt").decode("utf-8").splitlines():
        eng, pol = normalize(line)
        text_pairs.append((eng, pol))

In [6]:
# ================= CHANGED: hyperparamy =================
batch_size   = 1024
block_size   = 128      # musi >= max długość sekwencji w batchu
max_epochs   = 200
learning_rate= 1e-3
device       = 'cuda' if torch.cuda.is_available() else 'cpu'
n_embd       = 64
n_head       = 1
n_layer      = 4
dropout      = 0.0
torch.manual_seed(1337)

In [7]:
tok = pl_tokenizer
vocab_size = tok.get_vocab_size()
pad_id = tok.token_to_id("[pad]")
bos_id = tok.token_to_id("[start]")
eos_id = tok.token_to_id("[end]")

In [8]:
from torch.utils.data import Dataset, DataLoader

class LMTranslationDataset(Dataset):
    def __init__(self, text_pairs):
        self.text_pairs = text_pairs
    def __len__(self):
        return len(self.text_pairs)
    def __getitem__(self, idx):
        eng, pol = self.text_pairs[idx]
        # pełna sekwencja do teacher forcing
        s = f"[start] EN: {eng}\nPL: {pol} [end]"
        return s

def collate_lm(batch):
    enc = tok.encode_batch(batch, add_special_tokens=True)  # jeśli post-processor dodaje BOS/EOS, OK
    ids = [e.ids[:block_size] for e in enc]                # przytnij do block_size
    # padding zapewnia tok.enable_padding; jeśli nie masz, zrób ręcznie
    x = torch.tensor([seq[:-1] for seq in ids], dtype=torch.long)  # input
    y = torch.tensor([seq[1:]  for seq in ids], dtype=torch.long)  # target (shift)
    return x, y

dataloader = DataLoader(
    LMTranslationDataset(text_pairs),
    batch_size=batch_size, shuffle=True,
    collate_fn=collate_lm
)


In [32]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class DecoderOnlyLM(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd, padding_idx=pad_id)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
        # (opcjonalnie) powiąż wagi head z embeddingiem: self.lm_head.weight = self.token_embedding_table.weight

    def forward(self, idx, targets=None):
        B, T = idx.shape
        pos = torch.arange(T, device=idx.device)  # [T]
        tok_emb = self.token_embedding_table(idx)           # [B,T,C]
        pos_emb = self.position_embedding_table(pos)[None]  # [1,T,C]
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)                            # [B,T,V]
        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)),
                                   targets.view(-1),
                                   ignore_index=pad_id)
        return logits, loss

    @torch.no_grad()
    def generate(self, idx, max_new_tokens=64, temperature=1.0, top_k=None, stop_id=None):
        self.eval()
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :] / temperature
            if top_k is not None:
                v, _ = torch.topk(logits, top_k)
                logits[logits < v[:, [-1]]] = -float('inf')
            probs = F.softmax(logits, dim=-1)
            next_id = torch.multinomial(probs, num_samples=1)
            idx = torch.cat([idx, next_id], dim=1)
            if stop_id is not None and next_id.item() == stop_id:
                break
        return idx

model = DecoderOnlyLM().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# ================= CHANGED: trening na DataLoaderze =================
for epoch in range(max_epochs):
    for xb, yb in dataloader:
        xb, yb = xb.to(device), yb.to(device)
        _, loss = model(xb, yb)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
    print(f"epoch {epoch:02d} | loss {loss.item():.4f}")

# ================= CHANGED: generowanie tłumaczenia =================
@torch.no_grad()
def translate(eng_sentence: str, max_new_tokens=64):
    prefix = f"[start] EN: {eng_sentence}\nPL:"
    ids = tok.encode(prefix, add_special_tokens=True).ids
    idx = torch.tensor([ids], dtype=torch.long, device=device)
    out = model.generate(idx, max_new_tokens=max_new_tokens, temperature=0.8, top_k=50, stop_id=eos_id)
    text = tok.decode(out[0].tolist())
    # (opcjonalnie) wytnij fragment po "PL:" i przed [end]
    return text

print( translate("run!") )

epoch 00 | loss 4.9913
epoch 01 | loss 4.1280
epoch 02 | loss 3.6581
epoch 03 | loss 3.4677
epoch 04 | loss 3.2362
epoch 05 | loss 3.1254
epoch 06 | loss 3.0177
epoch 07 | loss 2.9145
epoch 08 | loss 2.8476
epoch 09 | loss 2.7419
epoch 10 | loss 2.6730
epoch 11 | loss 2.6608
epoch 12 | loss 2.6457
epoch 13 | loss 2.5637
epoch 14 | loss 2.5304
epoch 15 | loss 2.4656
epoch 16 | loss 2.4263
epoch 17 | loss 2.4674
epoch 18 | loss 2.4014
epoch 19 | loss 2.3288
epoch 20 | loss 2.2874
epoch 21 | loss 2.2932
epoch 22 | loss 2.3121
epoch 23 | loss 2.2398
epoch 24 | loss 2.1978
epoch 25 | loss 2.2393
epoch 26 | loss 2.2093
epoch 27 | loss 2.1422
epoch 28 | loss 2.1578
epoch 29 | loss 2.0760
epoch 30 | loss 2.1332
epoch 31 | loss 2.0499
epoch 32 | loss 2.0916
epoch 33 | loss 2.1133
epoch 34 | loss 1.9859
epoch 35 | loss 2.0571
epoch 36 | loss 1.9836
epoch 37 | loss 2.0296
epoch 38 | loss 1.9672
epoch 39 | loss 1.9733
epoch 40 | loss 1.9870
epoch 41 | loss 1.9876
epoch 42 | loss 1.9274
epoch 43 | 

In [51]:
translate("hello! what is your name?")

' : hello! what is your name?: cześć, co jest twoje? '